In [1]:
pip install pyreadr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.0/417.0 kB 6.7 MB/s eta 0:00:00


In [2]:
! pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 42.7 MB/s eta 0:00:00


In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random
import time
import pandas as pd
import pyreadr

In [4]:
#from google.colab import drive
#drive.mount('/content/drive')
!wget https://github.com/guadag12/scraping-google-scholar/raw/main/articles_df_ministerial_replacement.rds
articles_df = pyreadr.read_r('articles_df_ministerial_replacement.rds')
articles_df = articles_df[None] # extract the pandas data frame

!wget https://github.com/guadag12/scraping-google-scholar/raw/main/authors_df_ministerial_replacement.rds

authors_df = pyreadr.read_r('authors_df_ministerial_replacement.rds') # also works for RData
authors_df = authors_df[None] # extract the pandas data frame


--2024-12-03 23:04:11--  https://github.com/guadag12/scraping-google-scholar/raw/main/articles_df_ministerial_replacement.rds
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/guadag12/scraping-google-scholar/main/articles_df_ministerial_replacement.rds [following]
--2024-12-03 23:04:12--  https://raw.githubusercontent.com/guadag12/scraping-google-scholar/main/articles_df_ministerial_replacement.rds
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11688 (11K) [application/octet-stream]
Saving to: ‘articles_df_ministerial_replacement.rds’

articles_df_ministe 100%[===================>]  11.41K  --.-

In [5]:
articles_df.head()

,Title,URL,Journal,Amount_Citation,Citation_URL,Author_1,Author_2,Author_3,Author_4
0,Replacing cabinet ministers: Patterns of minis...,https://www.cambridge.org/core/journals/americ...,- cambridge.org,349,https://scholar.google.com/scholar?cites=10520...,No proper name available,Celso Martínez Rivero,NaN,NaN
1,Cabinet reshuffles and ministerial drift,https://www.cambridge.org/core/journals/britis...,- cambridge.org,264,https://scholar.google.com/scholar?cites=87530...,Indridi H. Indridason,Kam C. (Johnny) Chan,NaN,NaN
2,Personal or party roots of civil service patro...,https://journals.sagepub.com/doi/abs/10.1177/0...,- journals.sagepub.com,32,https://scholar.google.com/scholar?cites=15803...,Katarina Staronova,Marek Rybar,NaN,NaN
3,Responsible government and ministerial respons...,https://www.cambridge.org/core/journals/canadi...,- cambridge.org,108,https://scholar.google.com/scholar?cites=67930...,No proper name available,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,No citation URL available,CAIRO VIDAL MARTINS,NaN,NaN,NaN


In [14]:
# Initialize the WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run in headless mode
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("start-maximized")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(options=options)

In [15]:
# Function to scrape a single page
def scrape_page(driver, url):
    driver.get(url)
    time.sleep(random.uniform(2, 5))  # Random sleep to mimic human behavior

    citations = []
    try:
        # Wait for the results to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "gs_r"))
        )
        results = driver.find_elements(By.CLASS_NAME, "gs_r")

        for result in results:
            try:
                title_element = result.find_element(By.CSS_SELECTOR, ".gs_rt a")
                title = title_element.text
                citation_url = title_element.get_attribute("href")
            except:
                title = None
                citation_url = None

            try:
                authors_info = result.find_element(By.CSS_SELECTOR, ".gs_a").text
                authors = authors_info.split("-")[0].strip()
                year = ''.join(filter(str.isdigit, authors_info.split("-")[-1]))
            except:
                authors = None
                year = None

            try:
                num_citations_element = result.find_element(By.CSS_SELECTOR, ".gs_fl a:nth-child(3)")
                num_citations = int(''.join(filter(str.isdigit, num_citations_element.text)))
            except:
                num_citations = 0

            citations.append({
                'URL_citacion': url,
                "Title": title,
                "Citation_URL": citation_url,
                "Authors": authors,
                "Year": year,
                "Number_of_Citations": num_citations
            })
    except Exception as e:
        print(f"Error scraping page: {url}\n{e}")

    return citations

In [16]:
def scrape_all_pages(base_url, num_pages):
    all_citations = []

    for page_num in range(num_pages):
        page_url = f"{base_url}&start={page_num * 10}"
        print(f"Scraping page: {page_url}")
        page_citations = scrape_page(driver, page_url)
        all_citations.extend(page_citations)
        time.sleep(random.uniform(5, 10))  # Random sleep to avoid detection

    return all_citations

In [17]:
import re

def get_num_pages(driver, base_url):
    """
    Determine the total number of pages for the given base URL.
    """
    driver.get(base_url)

    try:
        # Wait for the page to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "gs_ab_md"))
        )

        # Extract total results text
        total_results_element = driver.find_element(By.ID, "gs_ab_md")
        total_results_text = total_results_element.text

        # Use regex to extract the total number of results
        match = re.search(r"About (\d+)", total_results_text)
        if match:
            total_results = int(match.group(1))
        else:
            match = re.search(r"(\d+)", total_results_text)  # Fallback for exact numbers
            total_results = int(match.group(1)) if match else 0

        # Calculate the number of pages (10 results per page)
        num_pages = -(-total_results // 10)  # Ceiling division
        return num_pages

    except Exception as e:
        print(f"Could not determine the number of pages for URL: {base_url}\nError: {e}")
        return 1  # Default to 1 if unable to determine


In [18]:
first = 0
second = 10

articles_df_2 = articles_df
articles_df_2 = articles_df_2["Citation_URL"][first:second]
articles_df_2

,Citation_URL
0,https://scholar.google.com/scholar?cites=10520...
1,https://scholar.google.com/scholar?cites=87530...
2,https://scholar.google.com/scholar?cites=15803...
3,https://scholar.google.com/scholar?cites=67930...
4,No citation URL available
5,https://scholar.google.com/scholar?q=related:z...
6,https://scholar.google.com/scholar?cites=17536...
7,https://scholar.google.com/scholar?cites=82206...
8,https://scholar.google.com/scholar?cites=20151...
9,https://scholar.google.com/scholar?cites=17692...


In [19]:
master_citations = []  # Store all citations data
for i, base_url in enumerate(articles_df_2):
    # Skip rows with invalid or missing URLs
    if pd.isna(base_url) or base_url == "No citation URL available":
        continue

    # Print progress
    print(f"Processing URL {i + 1} of {len(articles_df_2)}: {base_url}")

    # Determine the number of pages dynamically
    num_pages = 5
    print(f"Number of pages for URL: {num_pages}")

    # Scrape data from all pages for the given URL
    citations_data = scrape_all_pages(base_url, num_pages)

    # Append to the master list
    master_citations.extend(citations_data)

# Convert to DataFrame and save to CSV
master_citations_df = pd.DataFrame(master_citations)
master_citations_df.to_csv("all_citations_data_0_10.csv", index=False)

# Close the driver
driver.quit()

Processing URL 1 of 10: https://scholar.google.com/scholar?cites=10520276894087223736&as_sdt=20000005&sciodt=0,21&hl=en&oe=ASCII
Number of pages for URL: 5
Scraping page: https://scholar.google.com/scholar?cites=10520276894087223736&as_sdt=20000005&sciodt=0,21&hl=en&oe=ASCII&start=0
Scraping page: https://scholar.google.com/scholar?cites=10520276894087223736&as_sdt=20000005&sciodt=0,21&hl=en&oe=ASCII&start=10
Scraping page: https://scholar.google.com/scholar?cites=10520276894087223736&as_sdt=20000005&sciodt=0,21&hl=en&oe=ASCII&start=20
Scraping page: https://scholar.google.com/scholar?cites=10520276894087223736&as_sdt=20000005&sciodt=0,21&hl=en&oe=ASCII&start=30
Scraping page: https://scholar.google.com/scholar?cites=10520276894087223736&as_sdt=20000005&sciodt=0,21&hl=en&oe=ASCII&start=40
Processing URL 2 of 10: https://scholar.google.com/scholar?cites=8753014759504830822&as_sdt=20000005&sciodt=0,21&hl=en&oe=ASCII
Number of pages for URL: 5
Scraping page: https://scholar.google.com/sch

In [12]:
master_citations.extend(citations_data)

# Convert to DataFrame and save to CSV
master_citations_df = pd.DataFrame(master_citations)
master_citations_df.to_csv("all_citations_data_108_111.csv", index=False)

# Close the driver
driver.quit()

In [ ]:
citations_data

[{'Title': 'Replacing cabinet ministers: Patterns of ministerial stability in parliamentary democracies',
  'Citation_URL': 'https://scholar.google.com/scholar?cluster=10520276894087223736&hl=en&as_sdt=20000005&sciodt=0,21',
  'Authors': None,
  'Year': None,
  'Number_of_Citations': 0},
 {'Title': 'Parliamentary control of coalition governments',
  'Citation_URL': 'https://www.annualreviews.org/content/journals/10.1146/annurev.polisci.10.071105.104340',
  'Authors': 'K Strøm, WC Müller, DM Smith',
  'Year': '',
  'Number_of_Citations': 225},
 {'Title': 'Presidents, parties, and prime ministers: How the separation of powers affects party organization and behavior',
  'Citation_URL': 'https://books.google.com/books?hl=en&lr=&id=PtQgsQzC080C&oi=fnd&pg=PR7&ots=L9l4zyORG9&sig=E8M8QrXC5G3OpHoqVa4CaikhhbM',
  'Authors': 'DJ Samuels, MS Shugart',
  'Year': '',
  'Number_of_Citations': 893},
 {'Title': 'Are digital business and digital public services a driver for better energy security? Evide

In [ ]:
df

,Title,Citation_URL,Authors,Year,Number_of_Citations
0,Cabinet reshuffles and ministerial drift,https://scholar.google.com/scholar?cluster=875...,None,None,0
1,The duration and durability of cabinet ministers,https://journals.sagepub.com/doi/abs/10.1177/0...,"J Fischer, K Dowding, P Dumont",,117
2,"Coalition agreements, issue attention, and cab...",https://journals.sagepub.com/doi/abs/10.1177/0...,"H Klüver, H Bäck",,97
3,"Political parties, parliaments and legislative...",https://books.google.com/books?hl=en&lr=&id=po...,"H Bäck, M Debus",,145
4,The selection of ministers in Europe,https://api.taylorfrancis.com/content/books/mo...,"KM Dowding, P Dumont",,246
5,Drawing from the bargaining pool: Determinants...,https://journals.sagepub.com/doi/abs/10.1177/1...,"J Fleischer, M Seyfried",,57
6,Updating the Party Government data set,https://www.sciencedirect.com/science/article/...,"K Seki, LK Williams",,152
7,Women in presidential cabinets: Power players ...,https://books.google.com/books?hl=en&lr=&id=2J...,MC Escobar,,117
8,Designing cabinets: presidential politics and ...,https://journals.sagepub.com/doi/abs/10.1177/1...,C Martínez,,167
9,Gender and representation in Latin America,https://books.google.com/books?hl=en&lr=&id=LA...,LA Schwindt,,94
